In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import experiment.metrics.metric as metric
from experiment.metrics.metric import cosine_similarity

In [2]:
""" set default device to mps """

device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device

device(type='mps')

In [28]:
""" Rotary Position Encoding Test
1) integrate with multi-head attention
2) multiply it before separate the heads
최대한 행렬로 처리할 방법을 찾자
dim 0, 1이 같은 공간으로 들어가고, 
dim 2, 3이 같은 공간으로 들어가게

cos m*theta1, -sin m*theta1 0 0 0 0 0 0 0 0 0 0   여기가 그럼 디
sin m*theta1, cos m*theta1  0 0 0 0 0 0 0 0 0 0
0               0


[Rotary Postion Encoding Algorithm]
1) create a scaler tensor
2) make theta tensor
3) multiple theta tensor and scaler => m*theta1
4) linear projection to transformation matrix
5) multiply it with word embedding
5) linear projectino to query, key
    - value matrix does not multiply with transformation matrix
"""
BATCH_SIZE = 2
MAX_SEQ = 4
DIM_MODEL = 6
scaler = torch.arange(1, MAX_SEQ+1, device=device, dtype=torch.float).unsqueeze(1).repeat(1, DIM_MODEL).reshape(MAX_SEQ, DIM_MODEL).unsqueeze(0).repeat(BATCH_SIZE, 1, 1)
print(f"scaler tensor: {scaler, scaler.shape}")

query = torch.rand(BATCH_SIZE, MAX_SEQ, DIM_MODEL, device=device)
test = torch.mul(query, scaler)
query, test, test.shape

scaler tensor: (tensor([[[1., 1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3., 3.],
         [4., 4., 4., 4., 4., 4.]],

        [[1., 1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3., 3.],
         [4., 4., 4., 4., 4., 4.]]], device='mps:0'), torch.Size([2, 4, 6]))


(tensor([[[0.7860, 0.2183, 0.0284, 0.8639, 0.9899, 0.2814],
          [0.3021, 0.8276, 0.5825, 0.1848, 0.3665, 0.2139],
          [0.3881, 0.3099, 0.4308, 0.7325, 0.7994, 0.2794],
          [0.9599, 0.7172, 0.5006, 0.7459, 0.8299, 0.3577]],
 
         [[0.4471, 0.5927, 0.4665, 0.2697, 0.3125, 0.4082],
          [0.8227, 0.2622, 0.3222, 0.1226, 0.2020, 0.4608],
          [0.2831, 0.4174, 0.4632, 0.9952, 0.1790, 0.6835],
          [0.8030, 0.2301, 0.2727, 0.1897, 0.2957, 0.5198]]], device='mps:0'),
 tensor([[[0.7860, 0.2183, 0.0284, 0.8639, 0.9899, 0.2814],
          [0.6042, 1.6552, 1.1650, 0.3695, 0.7330, 0.4279],
          [1.1644, 0.9297, 1.2924, 2.1975, 2.3982, 0.8382],
          [3.8397, 2.8687, 2.0023, 2.9838, 3.3198, 1.4308]],
 
         [[0.4471, 0.5927, 0.4665, 0.2697, 0.3125, 0.4082],
          [1.6453, 0.5244, 0.6445, 0.2451, 0.4039, 0.9216],
          [0.8492, 1.2521, 1.3897, 2.9856, 0.5371, 2.0506],
          [3.2118, 0.9202, 1.0909, 0.7589, 1.1827, 2.0793]]], device='mps:0

In [29]:
""" make theta tensor """
i_arr = torch.arange(1, int(DIM_MODEL/2)+1).to(device)
i_arr = i_arr.repeat_interleave(2)
theta = 10000**(-2*(i_arr - 1)/DIM_MODEL)
theta, theta.shape

(tensor([1.0000, 1.0000, 0.0464, 0.0464, 0.0022, 0.0022], device='mps:0'),
 torch.Size([6]))

In [30]:
""" multiple scaler and theta tensor """
var_tensor = torch.mul(scaler, theta)
var_tensor, var_tensor.shape

(tensor([[[1.0000e+00, 1.0000e+00, 4.6416e-02, 4.6416e-02, 2.1544e-03,
           2.1544e-03],
          [2.0000e+00, 2.0000e+00, 9.2832e-02, 9.2832e-02, 4.3089e-03,
           4.3089e-03],
          [3.0000e+00, 3.0000e+00, 1.3925e-01, 1.3925e-01, 6.4633e-03,
           6.4633e-03],
          [4.0000e+00, 4.0000e+00, 1.8566e-01, 1.8566e-01, 8.6177e-03,
           8.6177e-03]],
 
         [[1.0000e+00, 1.0000e+00, 4.6416e-02, 4.6416e-02, 2.1544e-03,
           2.1544e-03],
          [2.0000e+00, 2.0000e+00, 9.2832e-02, 9.2832e-02, 4.3089e-03,
           4.3089e-03],
          [3.0000e+00, 3.0000e+00, 1.3925e-01, 1.3925e-01, 6.4633e-03,
           6.4633e-03],
          [4.0000e+00, 4.0000e+00, 1.8566e-01, 1.8566e-01, 8.6177e-03,
           8.6177e-03]]], device='mps:0'),
 torch.Size([2, 4, 6]))

In [49]:
BATCH_SIZE = 2
MAX_SEQ = 4
DIM_MODEL = 6
scaler = torch.arange(1, MAX_SEQ+1, device=device, dtype=torch.float).unsqueeze(1).repeat(1, DIM_MODEL).reshape(MAX_SEQ, DIM_MODEL).unsqueeze(0).repeat(BATCH_SIZE, 1, 1)

""" make theta tensor """
i_arr = torch.arange(1, int(DIM_MODEL/2)+1).to(device)
i_arr = i_arr.repeat_interleave(2)
theta = 10000**(-2*(i_arr - 1)/DIM_MODEL)
var_tensor = torch.mul(scaler, theta)


def create_rotation_matrix_v2(d, thetas):
    """
    Create a batch of rotation matrices from the given thetas.
    
    Args:
    - d (int): The dimensionality of the rotation matrix (must be even).
    - thetas (Tensor): A tensor of shape (batch_size, seq_len, d/2) containing the rotation angles.
    
    Returns:
    - Tensor: A tensor of shape (batch_size, seq_len, d, d) containing the rotation matrices.
    """
    # Validate the input dimensions
    batch_size, seq_len, _ = thetas.size()
    R = torch.eye(d, device=thetas.device).repeat(batch_size, seq_len, 1, 1)
    
    for i in range(0, d, 2):
        cos_t = torch.cos(thetas[:, :, i]).unsqueeze(-1)
        sin_t = torch.sin(thetas[:, :, i]).unsqueeze(-1)
    
        R[:, :, i, i] = cos_t.squeeze(-1)
        R[:, :, i+1, i+1] = cos_t.squeeze(-1)
        R[:, :, i, i+1] = -sin_t.squeeze(-1)
        R[:, :, i+1, i] = sin_t.squeeze(-1)
        
    return R
R = create_rotation_matrix_v2(DIM_MODEL, var_tensor)
var_tensor, var_tensor.shape, R, R.shape

(tensor([[[1.0000e+00, 1.0000e+00, 4.6416e-02, 4.6416e-02, 2.1544e-03,
           2.1544e-03],
          [2.0000e+00, 2.0000e+00, 9.2832e-02, 9.2832e-02, 4.3089e-03,
           4.3089e-03],
          [3.0000e+00, 3.0000e+00, 1.3925e-01, 1.3925e-01, 6.4633e-03,
           6.4633e-03],
          [4.0000e+00, 4.0000e+00, 1.8566e-01, 1.8566e-01, 8.6177e-03,
           8.6177e-03]],
 
         [[1.0000e+00, 1.0000e+00, 4.6416e-02, 4.6416e-02, 2.1544e-03,
           2.1544e-03],
          [2.0000e+00, 2.0000e+00, 9.2832e-02, 9.2832e-02, 4.3089e-03,
           4.3089e-03],
          [3.0000e+00, 3.0000e+00, 1.3925e-01, 1.3925e-01, 6.4633e-03,
           6.4633e-03],
          [4.0000e+00, 4.0000e+00, 1.8566e-01, 1.8566e-01, 8.6177e-03,
           8.6177e-03]]], device='mps:0'),
 torch.Size([2, 4, 6]),
 tensor([[[[ 0.5403, -0.8415,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.8415,  0.5403,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.9989, -0.0464,  0.0000,  0.

In [50]:
query = torch.rand(BATCH_SIZE, MAX_SEQ, DIM_MODEL, device=device)
query

tensor([[[0.5181, 0.0157, 0.2816, 0.8102, 0.7531, 0.3929],
         [0.2677, 0.4532, 0.2546, 0.8741, 0.9514, 0.7601],
         [0.8453, 0.2444, 0.2729, 0.9591, 0.9617, 0.2893],
         [0.7103, 0.3587, 0.7432, 0.9069, 0.2104, 0.2306]],

        [[0.7296, 0.3562, 0.6634, 0.2245, 0.1866, 0.4317],
         [0.5114, 0.0046, 0.2526, 0.8282, 0.1672, 0.7417],
         [0.1639, 0.5335, 0.7061, 0.8498, 0.0443, 0.5817],
         [0.5541, 0.2003, 0.7650, 0.3248, 0.2091, 0.3983]]], device='mps:0')

In [52]:
""" expand to hole word embedding shape """

for b in range(BATCH_SIZE):
    for s in range(MAX_SEQ):
        sub_rotary, sub_word = R[b, s, :, :], query[b, s, :]
        query[b, s, :] = torch.matmul(sub_rotary, sub_word)

In [53]:
class RoPE(nn.Module):
    def __init__(self, dim_model: int= 768):
        super().__init__()
        self.dim_model = dim_model
        self.i_arr = torch.arange(1, int(dim_model/2)+1) 
        self.theta = 10000**(-2*(self.i_arr - 1)/self.dim_model)
    
    def forward(self):
        print(self.theta.shape)
        print(self.theta)

tensor([[[ 0.2667,  0.4445,  0.2437,  0.8224,  0.7523,  0.3945],
         [-0.5235,  0.0548,  0.1725,  0.8940,  0.9481,  0.7642],
         [-0.8714, -0.1227,  0.1372,  0.9877,  0.9599,  0.2956],
         [-0.1929, -0.7720,  0.5630,  1.0285,  0.2084,  0.2324]],

        [[ 0.0945,  0.8064,  0.6523,  0.2550,  0.1857,  0.4321],
         [-0.2170,  0.4631,  0.1747,  0.8481,  0.1640,  0.7424],
         [-0.2375, -0.5051,  0.5813,  0.9395,  0.0405,  0.5820],
         [-0.2106, -0.5502,  0.6919,  0.4605,  0.2057,  0.4001]]],
       device='mps:0')

In [8]:
rotary_pos_enc = RoPE(DIM_MODEL)
rotary_pos_enc()

torch.Size([384])
tensor([1.0000e+00, 9.7630e-01, 9.5316e-01, 9.3057e-01, 9.0852e-01, 8.8699e-01,
        8.6596e-01, 8.4544e-01, 8.2540e-01, 8.0584e-01, 7.8674e-01, 7.6810e-01,
        7.4989e-01, 7.3212e-01, 7.1477e-01, 6.9783e-01, 6.8129e-01, 6.6515e-01,
        6.4938e-01, 6.3399e-01, 6.1897e-01, 6.0430e-01, 5.8997e-01, 5.7599e-01,
        5.6234e-01, 5.4901e-01, 5.3600e-01, 5.2330e-01, 5.1090e-01, 4.9879e-01,
        4.8697e-01, 4.7543e-01, 4.6416e-01, 4.5316e-01, 4.4242e-01, 4.3193e-01,
        4.2170e-01, 4.1170e-01, 4.0195e-01, 3.9242e-01, 3.8312e-01, 3.7404e-01,
        3.6517e-01, 3.5652e-01, 3.4807e-01, 3.3982e-01, 3.3177e-01, 3.2390e-01,
        3.1623e-01, 3.0873e-01, 3.0142e-01, 2.9427e-01, 2.8730e-01, 2.8049e-01,
        2.7384e-01, 2.6735e-01, 2.6102e-01, 2.5483e-01, 2.4879e-01, 2.4289e-01,
        2.3714e-01, 2.3152e-01, 2.2603e-01, 2.2067e-01, 2.1544e-01, 2.1034e-01,
        2.0535e-01, 2.0049e-01, 1.9573e-01, 1.9110e-01, 1.8657e-01, 1.8214e-01,
        1.7783e-01, 1.

In [8]:
""" test for parallel multi-head attention
1) permute. contiguous, reshape
2) directly call reshape from torch.matmul(attention_matrix, v)
"""

query = torch.rand(1, 8, 4, 2, device=device)
BS, SEQ_LEN, NUM_HEADS, DIM_HEADS = query.shape
print(f"original query: {query.shape, query}")

test1 = query.permute(0, 2, 1, 3).contiguous().reshape(-1, SEQ_LEN, NUM_HEADS*DIM_HEADS)
print(f"test1: {test1.shape, test1}")

test2 = query.reshape(-1, SEQ_LEN, NUM_HEADS*DIM_HEADS)
print(f"test2: {test2.shape, test2}")

original query: (torch.Size([1, 8, 4, 2]), tensor([[[[0.9168, 0.0523],
          [0.5834, 0.3648],
          [0.1445, 0.6178],
          [0.5795, 0.7004]],

         [[0.9812, 0.6302],
          [0.1581, 0.9039],
          [0.3351, 0.6687],
          [0.9043, 0.7900]],

         [[0.4702, 0.4620],
          [0.0865, 0.3290],
          [0.5967, 0.5650],
          [0.1954, 0.9916]],

         [[0.4202, 0.9388],
          [0.4909, 0.4016],
          [0.4052, 0.9085],
          [0.5089, 0.3139]],

         [[0.8943, 0.1367],
          [0.1901, 0.1161],
          [0.3471, 0.0353],
          [0.1799, 0.8890]],

         [[0.8990, 0.4810],
          [0.8155, 0.6152],
          [0.7975, 0.7668],
          [0.6558, 0.6093]],

         [[0.6960, 0.4698],
          [0.1492, 0.0688],
          [0.0338, 0.2450],
          [0.6533, 0.4803]],

         [[0.2576, 0.7272],
          [0.7371, 0.8030],
          [0.2122, 0.0413],
          [0.5253, 0.0612]]]], device='cuda:0'))
test1: (torch.Size([1, 8, 

In [13]:
""" source code from original linear transformers paper github
"""

max_seq = 5
dim_model = 8
num_heads = 4
dim_head = dim_model // num_heads
batch_size = 3

# 3, 5, 4, 2
query = torch.rand(batch_size, max_seq, num_heads, dim_head, device=device)
key = torch.rand(batch_size, max_seq, num_heads, dim_head, device=device)
value = torch.rand(batch_size, max_seq, num_heads, dim_head, device=device)

kv = torch.einsum("nshd,nshm->nhmd", key, value)
print(f"kv: {kv.shape, kv}")

z = 1/(torch.einsum("nlhd,nhd->nlh", query, key.sum(dim=1))+1e-6)
print(f"z: {z.shape, z}")

qkv = torch.einsum("nlhd,nhmd->nlhm", query, kv)
print(f"qkv: {qkv.shape, qkv}")

V = torch.einsum("nlhd,nhmd,nlh->nlhm", query, kv, z)
print(f"V: {V.shape, V}")

test_v = torch.einsum("nlhm,nlh->nlhm", qkv, z)
print(f"test_v: {test_v.shape, test_v}")

kv: (torch.Size([3, 4, 2, 2]), tensor([[[[1.8712, 1.7441],
          [2.0152, 1.8818]],

         [[1.2480, 1.6586],
          [0.9948, 1.8648]],

         [[1.8897, 1.1054],
          [1.4765, 1.0960]],

         [[1.1829, 2.3455],
          [1.1215, 1.7871]]],


        [[[1.2551, 1.4782],
          [0.6835, 0.8580]],

         [[2.0954, 2.2804],
          [1.2951, 1.9219]],

         [[1.5423, 1.0609],
          [2.4590, 1.7339]],

         [[2.1012, 1.5119],
          [1.1772, 0.9965]]],


        [[[1.3803, 2.0247],
          [1.0918, 1.3187]],

         [[1.6456, 1.3337],
          [0.9049, 1.3710]],

         [[1.0539, 1.3264],
          [1.2296, 1.8057]],

         [[1.3767, 1.8384],
          [0.7315, 0.7558]]]], device='mps:0'))
z: (torch.Size([3, 5, 4]), tensor([[[0.4404, 0.2922, 0.5909, 0.2315],
         [0.2317, 0.4030, 0.4618, 0.2901],
         [0.6622, 0.5419, 0.2251, 0.2437],
         [0.6661, 0.3618, 0.3576, 0.9171],
         [0.2153, 0.3808, 0.4363, 0.4122]],

       

In [114]:
""" alias of each tensor dimension for torch einsum
b: batch_size
s: sequence length
q: dim_head of query
k: dim_head of key
v: dim_head of value
"""
def kernel_fn(x: Tensor):
    return F.elu(x) + 1

query = torch.rand(batch_size, max_seq, dim_head, device=device)
key = torch.rand(batch_size, max_seq, dim_head, device=device)
value = torch.rand(batch_size, max_seq, dim_head, device=device)
padding_mask = torch.tensor([
    [0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1],
    [0, 0, 1, 1, 1]],
    device=device
)
query, key = kernel_fn(query), kernel_fn(key)
query[padding_mask == 1], key[padding_mask == 1], value[padding_mask == 1] = 0, 0, 0

KV = torch.matmul(value.permute(0, 2, 1), key.permute(0, 1, 2))
print(f"KV: {KV.shape, KV}")

Z = 1 / torch.clamp(torch.mul(query,key.sum(dim=1).unsqueeze(1)).sum(dim=-1), min=1e-6)
print(f"Z: {Z.shape, Z}")

V = torch.einsum("bsq,bvk,bs->bsv", query, KV, Z)
print(f"V: {V.shape, V}")

KV: (torch.Size([3, 2, 2]), tensor([[[1.6945, 2.5599],
         [1.6986, 2.4920]],

        [[3.7326, 3.1925],
         [3.5027, 3.1093]],

        [[1.4581, 2.1359],
         [2.1605, 2.8166]]], device='cuda:0'))
Z: (torch.Size([3, 5]), tensor([[5.9278e-02, 8.4242e-02, 8.2484e-02, 1.0000e+06, 1.0000e+06],
        [4.6290e-02, 4.3917e-02, 6.5639e-02, 6.5267e-02, 1.0000e+06],
        [1.0978e-01, 1.0496e-01, 1.0000e+06, 1.0000e+06, 1.0000e+06]],
       device='cuda:0'))
V: (torch.Size([3, 5, 2]), tensor([[[0.9854, 0.9707],
         [1.0248, 1.0095],
         [1.0131, 0.9979],
         [0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[1.1236, 1.0728],
         [1.1181, 1.0675],
         [1.1167, 1.0662],
         [1.1213, 1.0706],
         [0.0000, 0.0000]],

        [[1.0962, 1.5180],
         [1.1320, 1.5677],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000]]], device='cuda:0'))


In [15]:
# [bs, num_heads, dim_head, max_seq]*[bs, num_heads, max_seq, dim_head] = [bs, num_heads, dim_head, dim_head]
KV = torch.matmul(value.permute(0, 2, 3, 1), key.permute(0, 2, 1, 3))  
print(f"KV: {KV.shape, KV}")

print(f"query: {query.shape}")
print(f"key sum: {key.sum(dim=1).shape}")
 
Z = 1 / torch.clamp(torch.mul(query,key.sum(dim=1).unsqueeze(1)).sum(dim=-1), min=1e-6)
print(f"Z: {Z, Z.shape}")

KV: (torch.Size([3, 4, 2, 2]), tensor([[[[1.8712, 1.7441],
          [2.0152, 1.8818]],

         [[1.2480, 1.6586],
          [0.9948, 1.8648]],

         [[1.8897, 1.1054],
          [1.4765, 1.0960]],

         [[1.1829, 2.3455],
          [1.1215, 1.7871]]],


        [[[1.2551, 1.4782],
          [0.6835, 0.8580]],

         [[2.0954, 2.2804],
          [1.2951, 1.9219]],

         [[1.5423, 1.0609],
          [2.4590, 1.7339]],

         [[2.1012, 1.5119],
          [1.1772, 0.9965]]],


        [[[1.3803, 2.0247],
          [1.0918, 1.3187]],

         [[1.6456, 1.3337],
          [0.9049, 1.3710]],

         [[1.0539, 1.3264],
          [1.2296, 1.8057]],

         [[1.3767, 1.8384],
          [0.7315, 0.7558]]]], device='mps:0'))
query: torch.Size([3, 5, 4, 2])
key sum: torch.Size([3, 4, 2])
Z: (tensor([[[0.4404, 0.2922, 0.5909, 0.2315],
         [0.2317, 0.4030, 0.4618, 0.2901],
         [0.6622, 0.5419, 0.2251, 0.2437],
         [0.6661, 0.3618, 0.3576, 0.9171],
         [0.

In [5]:
# test_query = query.reshape(batch_size, max_seq, dim_head)
# test_key = key.reshape(batch_size, max_seq, dim_head)
# test_value = value.reshape(batch_size, max_seq, dim_head)
# 
# KV = torch.matmul(test_key.permute(0, 2, 1), test_value)
# print(f"KV: {KV.shape, KV}")
# QKV = torch.matmul(test_query, KV)
# print(f"QKV: {QKV.shape, QKV}")

# summation_key = test_key.sum(dim=1).unsqueeze(1)
# print(f"key, summation_key: {test_key.shape, summation_key.shape}")
# 
# Z = 1/torch.clamp(torch.mul(test_query, summation_key), min=1e-6)
# print(f"Normalizer Z: {Z, Z.shape}")
# 
# linear_attn_matrix = torch.matmul(QKV, Z)
# print(f"linear_attn_matrix: {linear_attn_matrix.shape, linear_attn_matrix}")

RuntimeError: shape '[3, 5, 2]' is invalid for input of size 120

In [84]:
quadratic = 512*64*64*512 + 512*512*512*64
linear = 64*512*512*64 + 512*64*64*64

quadratic, linear, quadratic // linear

(9663676416, 1207959552, 8)

In [98]:
""" Inputs for Linear Attentions
"""

max_seq = 5
dim_model = 2
num_heads = 12
batch_size = 3
# dim_head = dim_model // num_heads
dim_head = dim_model

def kernel_fn(x: Tensor):
    return F.elu(x) + 1

query = torch.rand(batch_size, max_seq, dim_head, device=device)
key = torch.rand(batch_size, max_seq, dim_head, device=device)
value = torch.rand(batch_size, max_seq, dim_head, device=device)

padding_mask = torch.tensor([
    [0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1],
    [0, 0, 1, 1, 1]],
    device=device
)
query.shape, key.shape, value.shape, padding_mask.shape

(torch.Size([3, 5, 2]),
 torch.Size([3, 5, 2]),
 torch.Size([3, 5, 2]),
 torch.Size([3, 5]))

In [99]:
""" Testing for Linear Attentions: KV Matrix
"""
k_query, k_key = kernel_fn(query), kernel_fn(key)
k_query[padding_mask == 1], k_key[padding_mask == 1], value[padding_mask == 1] = 0, 0, 0 

KV = torch.matmul(k_key.permute(0, 2, 1), value)
KV, KV.shape

(tensor([[[1.6423, 2.3725],
          [1.9186, 2.7227]],
 
         [[2.5099, 4.3219],
          [2.0842, 3.6178]],
 
         [[1.7249, 2.7095],
          [1.6283, 2.5576]]], device='cuda:0'),
 torch.Size([3, 2, 2]))

In [100]:
""" Testing for Linear Attentions: QKV
"""

QKV = torch.matmul(k_query, KV)
QKV, QKV.shape

(tensor([[[ 6.9833,  9.9921],
          [ 4.8467,  6.9438],
          [ 5.7299,  8.2035],
          [ 0.2364,  0.8221],
          [ 0.5723,  0.9037]],
 
         [[ 6.3969, 11.0627],
          [ 6.1451, 10.6128],
          [ 6.6618, 11.5190],
          [ 7.9548, 13.7416],
          [ 0.5258,  0.1820]],
 
         [[ 4.7064,  7.3925],
          [ 4.8010,  7.5412],
          [ 0.4323,  0.3144],
          [ 0.6062,  0.6203],
          [ 0.8389,  0.5675]]], device='cuda:0'),
 torch.Size([3, 5, 2]))

In [101]:
""" Testing for Linear Attentions: QKV / normalizer Z
softmax는 row-wise하게 정규화 하는데, 우리도 똑같이 정규화가 필요하지 않냐고 그래서 Z가 필요
여기서, QKV가 결국 row-wise 하게 정규화 되어야 한다는게 포인트임
그렇다면 Z의 크기는 16, 512, 64가 되어야 한다
"""

summation_key = k_key.sum(dim=1).unsqueeze(1).expand(-1, dim_head, -1).permute(0,2,1)

print(f"key, summation_key: {key.shape, summation_key.shape}")

Z = 1/torch.clamp(torch.matmul(k_query, summation_key), min=1e-6)
print(f"Normalizer Z: {Z, Z.shape}")

key, summation_key: (torch.Size([3, 5, 2]), torch.Size([3, 2, 2]))
Normalizer Z: (tensor([[[5.6876e-02, 5.6876e-02],
         [8.1770e-02, 8.1770e-02],
         [6.9246e-02, 6.9246e-02],
         [1.0000e+06, 1.0000e+06],
         [1.0000e+06, 1.0000e+06]],

        [[6.3809e-02, 6.3809e-02],
         [6.7458e-02, 6.7458e-02],
         [6.1383e-02, 6.1383e-02],
         [5.1966e-02, 5.1966e-02],
         [1.0000e+06, 1.0000e+06]],

        [[9.9465e-02, 9.9465e-02],
         [9.7511e-02, 9.7511e-02],
         [1.0000e+06, 1.0000e+06],
         [1.0000e+06, 1.0000e+06],
         [1.0000e+06, 1.0000e+06]]], device='cuda:0'), torch.Size([3, 5, 2]))


In [102]:
""" Testing for Linear Attentions: QKV / normalizer Z 
"""

linear_attn_matrix = torch.mul(QKV, Z)
print(f"linear_attn_matrix: {linear_attn_matrix.shape, linear_attn_matrix}")

linear_attn_matrix: (torch.Size([3, 5, 2]), tensor([[[3.9718e-01, 5.6831e-01],
         [3.9631e-01, 5.6779e-01],
         [3.9677e-01, 5.6806e-01],
         [2.3637e+05, 8.2210e+05],
         [5.7225e+05, 9.0375e+05]],

        [[4.0818e-01, 7.0590e-01],
         [4.1454e-01, 7.1592e-01],
         [4.0892e-01, 7.0707e-01],
         [4.1338e-01, 7.1410e-01],
         [5.2581e+05, 1.8200e+05]],

        [[4.6812e-01, 7.3529e-01],
         [4.6815e-01, 7.3535e-01],
         [4.3231e+05, 3.1439e+05],
         [6.0615e+05, 6.2035e+05],
         [8.3891e+05, 5.6749e+05]]], device='cuda:0'))


In [103]:
""" Comparing with pure self-attention
"""

attn_matrix = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(dim_head))
print(f"attn_matrix: {attn_matrix.shape, attn_matrix}")

padding_mask = padding_mask.unsqueeze(1)
attn_matrix = attn_matrix.masked_fill(padding_mask == 1, float('-inf'))
attention_dist = F.softmax(attn_matrix, dim=-1)
print(f"attention_dist: {attention_dist.shape, attention_dist}")

attention_matrix = torch.matmul(attention_dist, value)
print(f"attention_matrix: {attention_matrix.shape, attention_matrix}")

attn_matrix: (torch.Size([3, 5, 5]), tensor([[[0.3713, 0.6363, 1.0094, 0.5870, 0.7410],
         [0.1055, 0.2372, 0.3701, 0.2420, 0.2239],
         [0.2164, 0.4022, 0.6346, 0.3840, 0.4393],
         [0.3069, 0.4024, 0.6519, 0.3202, 0.5834],
         [0.1555, 0.2623, 0.4167, 0.2403, 0.3093]],

        [[0.2487, 0.1702, 0.1442, 0.2789, 0.3663],
         [0.0394, 0.2411, 0.1327, 0.3839, 0.3884],
         [0.2519, 0.2133, 0.1670, 0.3475, 0.4341],
         [0.2112, 0.4541, 0.2813, 0.7280, 0.7887],
         [0.0527, 0.1607, 0.0945, 0.2568, 0.2699]],

        [[0.3726, 0.5146, 0.2369, 0.3336, 0.5193],
         [0.4214, 0.5751, 0.1496, 0.1591, 0.4101],
         [0.7383, 1.0163, 0.4114, 0.5539, 0.9420],
         [0.6791, 0.9325, 0.3396, 0.4380, 0.8084],
         [0.5018, 0.6880, 0.2337, 0.2919, 0.5716]]], device='cuda:0'))
attention_dist: (torch.Size([3, 5, 5]), tensor([[[0.2383, 0.3106, 0.4511, 0.0000, 0.0000],
         [0.2904, 0.3313, 0.3783, 0.0000, 0.0000],
         [0.2686, 0.3234, 0.4080

In [25]:
""" Comparing with pure self-attention by KL-divergence
배치 별, 평균 총합 0.5 ~ 2 정도 차이남, 이게 보니까 처음에 랜덤 초기화 빨로 갈리네
"""

kl_div = F.kl_div(linear_attn_matrix.log(), attention_matrix, reduction='batchmean')
kl_div

tensor(inf, device='cuda:0')

In [33]:
""" Test code for applying padding masking to linear attention """
test_q = torch.randn(3, 5, 4, device=device)
test_k = torch.randn(3, 5, 4, device=device)
test_v = torch.randn(3, 5, 4, device=device)

padding_mask = torch.tensor([
    [0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1],
    [0, 0, 1, 1, 1]],
    device=device
)
test_k[padding_mask == 1] = 0
test_k

tensor([[[-0.0264, -0.0698, -1.3971, -1.5667],
         [ 0.5706, -1.4718,  0.3398, -0.5280],
         [ 1.5465,  0.2453,  1.0911,  0.6062],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.4773,  1.9033,  0.6571,  0.4388],
         [ 0.1047, -0.3162, -3.4738, -0.4424],
         [ 0.4799,  0.7670, -1.0056, -0.4248],
         [ 1.2279, -0.7639,  1.4043, -0.2604],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.1115,  0.3349, -1.0625,  0.5592],
         [ 0.3361,  1.7460,  1.9226,  0.5757],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], device='cuda:0')

In [92]:
class RoPE(nn.Module):
    def __init__(self, dim_model: int= 768):
        super().__init__()
        self.dim_model = dim_model
        self.i_arr = torch.arange(1, int(dim_model/2)+1)  # 세타값을 살리려면 
        self.theta = 10000**(-2*(self.i_arr - 1)/self.dim_model)
    
    def forward(self):
        print(self.i_arr.shape)
        print(self.i_arr)
        print(self.theta.shape)
        print(self.theta)

In [93]:
test = RoPE()
test()

torch.Size([384])
tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
         85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
         99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176